<a href="https://colab.research.google.com/github/godhunter98/Linear_models/blob/main/Titanic_Dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# import kaggle

In [ ]:
# kaggle.api.competition_download_cli(competition="titanic")

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
train_data = pd.read_csv('https://github.com/godhunter98/Titanic/blob/f33102a1329c36adcbd3928c163ddd41e322d275/train.csv?raw=True')

In [ ]:
train_data.shape

(891, 12)

In [ ]:
train_data.columns

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')

In [ ]:
train_data.dtypes

,0
PassengerId,int64
Survived,int64
Pclass,int64
Name,object
Sex,object
Age,float64
SibSp,int64
Parch,int64
Ticket,object
Fare,float64


In [ ]:
train_data.drop('Cabin',inplace=True,axis=1)

In [ ]:
X = train_data.copy()
y = X.pop('Survived')

In [ ]:
# let's seperate our numerical and Cat Columns as the preprocessing steps for both will be different
num_cols = [cols for cols in X.columns if X[cols].dtype in ['int64','float64']]
cat_cols = [cols for cols in X.columns if X[cols].dtype == 'object']

In [ ]:
# time to split our data into training and validation set
# Note-- we don't do any preproccesing/tranformation befor splitting as we don't want any data leakage issue
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.25)

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

In [ ]:
# Preprocessing for numerical data
numerical_transformer = SimpleImputer(strategy='constant')

# Preprocessing for categorical data
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

In [ ]:
preprocessor= ColumnTransformer(
    [('num',numerical_transformer,num_cols),
     ('cat',categorical_transformer,cat_cols)]
    )

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier

In [ ]:
# xgb_model = XGBClassifier(n_estimators=500, learning_rate=0.1, early_stopping_rounds=10)

In [ ]:
# model_pipeline = Pipeline(steps=[
#     ('preprocessor', preprocessor),
#     ('model', xgb_model)
# ])

In [ ]:
model_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('model', RandomForestClassifier(n_estimators=500, max_depth=6, random_state=1))
])

In [ ]:
model_pipeline.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  SimpleImputer(strategy='constant'),
                                                  ['PassengerId', 'Pclass',
                                                   'Age', 'SibSp', 'Parch',
                                                   'Fare']),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  ['Name', 'Sex', 'Ticket',
                                                   'Embarked'])])),
                ('model',
                 RandomForestClassifier(max_depth=6, n_estimators=500,
                                        random_state=1))])

In [ ]:
# X_train_preprocessed = preprocessor.fit_transform(X_train)
# X_test_preprocessed = preprocessor.transform(X_test)

In [ ]:
# model_pipeline.fit(X_train, y_train, model__eval_set=[(X_test_preprocessed, y_test)])

In [ ]:
print(model_pipeline.score(X_test,y_test))

0.8071748878923767


In [ ]:
test_data = pd.read_csv('https://github.com/godhunter98/Titanic/blob/f33102a1329c36adcbd3928c163ddd41e322d275/test.csv?raw=True')

In [ ]:
test_data.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [ ]:
x = [(x, y) for x, y in zip(test_data['PassengerId'], model_pipeline.predict(test_data))]

In [ ]:
x = pd.DataFrame(x,columns=['PassengerId','Survived'])

In [ ]:
x.to_csv('submission.csv',index=False)

In [ ]:
!pwd

/content
